In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf", from_type="gemma", load_eager=True, device_map="tpu:0")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
from sprint.icl_sfc_utils import Circuitizer

In [6]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [7]:
task_names = ["en_es", "antonyms", "person_profession"]
task_name = task_names[0]
task_name = "antonyms"

In [8]:
def check_if_single_token(token):
    return len(tokenizer.tokenize(token)) == 1

task = tasks[task_name]

print(len(task))

# task = {
#     k:v for k,v in task.items() if check_if_single_token(k) and check_if_single_token(v)
# }

print(len(task))

pairs = list(task.items())

batch_size = 8 
n_shot=16
if task_name.startswith("algo"):
    n_shot = 8
max_seq_len = 128
seed = 10

prompt = "Follow the pattern:\n{}"

runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=n_shot, max_seq_len=max  _seq_len, seed=seed, prompt=prompt, use_same_examples=True, use_same_target=False)


layers = list(range(11, 17))
circuitizer = Circuitizer(llama, tokenizer, runner, layers, prompt)

162
162
Setting up masks...
Running metrics...
Setting up RMS...


  0%|          | 0/18 [00:00<?, ?it/s]

Loading SAEs...


  0%|          | 0/6 [00:00<?, ?it/s]

Running node IEs...


  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [1]:

layers = [10,11,12]
orig_metric = circuitizer.ablated_metric(llama, runner=(runner, tokenizer)).tolist()
zero_metric = circuitizer.run_ablated_metrics([100000], layers=layers, runner=(runner, tokenizer), prompt=prompt)[0][0]

print(orig_metric, zero_metric)

NameError: name 'circuitizer' is not defined

In [2]:
import numpy as np
# thresholds = np.linspace(0, 1e-4, 100)
# thresholds = np.linspace(1.4 * 1e-4, 1.45 * 1e-4, 200)
thresholds = np.logspace(-4, -1, 150)
topks = [4, 6, 12, 16, 24, 32]

inverse = False
do_abs = False
mean_ablate = False
average_over_positions = True


ablated_metrics, n_nodes_counts = circuitizer.run_ablated_metrics(thresholds, inverse=inverse, 
                                                                  do_abs=do_abs, mean_ablate=mean_ablate, 
                                                                  average_over_positions=average_over_positions,
                                                                  token_prefix=None, layers=layers)

faithfullness = np.array(ablated_metrics)
faithfullness = (faithfullness - zero_metric) / (orig_metric - zero_metric)



# target_metric = (max(ablated_metrics) - min(ablated_metrics)) * 0.95 + min(ablated_metrics)
# target_threshold = [threshold for threshold, metric in reversed(list(zip(thresholds, ablated_metrics))) if metric > target_metric][0]

NameError: name 'circuitizer' is not defined

In [14]:
n_nodes_counts

[6734,
 6619,
 6507,
 6394,
 6285,
 6173,
 6058,
 5962,
 5861,
 5749,
 5637,
 5523,
 5446,
 5350,
 5258,
 5144,
 5032,
 4954,
 4854,
 4743,
 4646,
 4526,
 4414,
 4306,
 4209,
 4101,
 4002,
 3897,
 3790,
 3694,
 3605,
 3519,
 3431,
 3355,
 3267,
 3177,
 3084,
 2982,
 2885,
 2808,
 2707,
 2615,
 2533,
 2453,
 2363,
 2299,
 2231,
 2153,
 2083,
 2013,
 1935,
 1870,
 1805,
 1724,
 1654,
 1606,
 1547,
 1488,
 1439,
 1374,
 1325,
 1268,
 1210,
 1161,
 1115,
 1072,
 1032,
 988,
 943,
 907,
 873,
 835,
 798,
 761,
 724,
 712,
 677,
 654,
 627,
 596,
 569,
 542,
 516,
 492,
 468,
 443,
 420,
 398,
 375,
 351,
 331,
 313,
 298,
 281,
 256,
 246,
 234,
 227,
 219,
 211,
 203,
 194,
 186,
 175,
 167,
 159,
 151,
 144,
 137,
 129,
 121,
 116,
 104,
 98,
 91,
 83,
 79,
 75,
 69,
 66,
 62,
 59,
 58,
 53,
 51,
 50,
 43,
 40,
 40,
 38,
 35,
 34,
 32,
 29,
 28,
 23,
 22,
 20,
 18,
 13,
 12,
 10,
 10,
 8,
 8,
 7,
 4,
 3,
 2,
 2]

In [20]:
n_nodes_counts

[6734,
 6619,
 6507,
 6394,
 6285,
 6173,
 6058,
 5962,
 5861,
 5749,
 5637,
 5523,
 5446,
 5350,
 5258,
 5144,
 5032,
 4954,
 4854,
 4743,
 4646,
 4526,
 4414,
 4306,
 4209,
 4101,
 4002,
 3897,
 3790,
 3694,
 3605,
 3519,
 3431,
 3355,
 3267,
 3177,
 3084,
 2982,
 2885,
 2808,
 2707,
 2615,
 2533,
 2453,
 2363,
 2299,
 2231,
 2153,
 2083,
 2013,
 1935,
 1870,
 1805,
 1724,
 1654,
 1606,
 1547,
 1488,
 1439,
 1374,
 1325,
 1268,
 1210,
 1161,
 1115,
 1072,
 1032,
 988,
 943,
 907,
 873,
 835,
 798,
 761,
 724,
 712,
 677,
 654,
 627,
 596,
 569,
 542,
 516,
 492,
 468,
 443,
 420,
 398,
 375,
 351,
 331,
 313,
 298,
 281,
 256,
 246,
 234,
 227,
 219,
 211,
 203,
 194,
 186,
 175,
 167,
 159,
 151,
 144,
 137,
 129,
 121,
 116,
 104,
 98,
 91,
 83,
 79,
 75,
 69,
 66,
 62,
 59,
 58,
 53,
 51,
 50,
 43,
 40,
 40,
 38,
 35,
 34,
 32,
 29,
 28,
 23,
 22,
 20,
 18,
 13,
 12,
 10,
 10,
 8,
 8,
 7,
 4,
 3,
 2,
 2]

In [24]:
import matplotlib.pyplot as plt
import plotly.express as px

# plt.plot([max(n_nodes_counts) - x for x in n_nodes_counts], ablated_metrics)
# plt.plot(thresholds, ablated_metrics)
# plt.plot(thresholds, ablated_metrics)
# plt.plot(n_nodes_counts, ablated_metrics)
# plt.plot(n_nodes_counts, faithfullness)
# plt.xscale("log")
# plt.plot(n_nodes_counts)

# px.line(x=list(range(len(ablated_metrics))), y=ablated_metrics)
# fig = px.line(x=thresholds, y=ablated_metrics)
# fig.update_xaxes(type="log", exponentformat="power")

fig = px.line(x=n_nodes_counts, y=faithfullness, title=f"inverse={inverse}, abs={do_abs}, mean={mean_ablate}, aop={average_over_positions}, layers={layers}")
fig.update_xaxes(title="Number of nodes")
fig.update_yaxes(title="Faithfullness")

# fig


In [12]:
target_faithfullness = 0.6

target_threshold = [threshold for threshold, metric in reversed(list(zip(thresholds, faithfullness))) if metric > target_faithfullness][0]

target_threshold

0.00984665202794123